Estoy tratando de recuperar la info cruda del final del experimento que hice para medir la dependencia con el umbral. 

De los registros de consentimiento que tengo anotados figura que los datos los tome entre el 29/3/16 y el 14/4/16

La idea es ir a backups de los db.json y volver a procesar todo rapido con la experiencia que ya tengo acumulada para hacer graficos diferentes a los que tengo guardados que son horribles. 

No tengo muy fresco como era el setup experimental ni la estructura de datos guardados, pero voy a ver que puedo hacer al respecto. Deberia tener al menos registro de 14 usuarios. 

Copio a la carpeta data todos los archivos json que corresponden a esas fechas. Quiero un codigo que busque todos los archivos y recupere la info util

Acabo de encontrar en este archivo que los sujetos completaron una encuesta cuando terminaron y tiene marcas temporales!! :)

https://docs.google.com/spreadsheets/d/1hV9QL78TFX2Ka4ROaAg9mHZ2WgcUDArZM2W8W_15wkg/edit#gid=1050318047



In [2]:
# Primero queremos listas todos los archivos

from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir('./data/') if isfile(join('./data/', f))]

In [47]:
# Queremos transformar cada archivo en un dataframe con etradas para ver entradas repetidas

import pandas as pd
import json

#Creamos la lista vacia de envios

sesiones = []
levels = []
convergencias = []

#Buscamos leer todas las entradas
for file in onlyfiles:
    with open('./data/'+file) as data_file:
            db = json.load(data_file)
    for envio in db['Envio']:
        if envio['tipoDeEnvio'] == 'NEWSESION':
            sesiones = sesiones + [envio]
        elif envio['tipoDeEnvio'] == 'NEWLEVEL':
            levels = levels + [envio]
        elif envio['tipoDeEnvio'] == 'CONVERGENCIA':
            convergencias = convergencias + [envio]
        elif envio['tipoDeEnvio']:
            print (envio['tipoDeEnvio'])
        else: 
            print (envio)

# Filtramos por envios unicos
envioshechos = []

sesionesUnique = []
for envio in sesiones:
    if not envio['instance'] in envioshechos:
        sesionesUnique = sesionesUnique + [json.loads(envio['contenido'])]
        envioshechos = envioshechos + [envio['instance']]

levelsUnique = []
for envio in levels:
    if not envio['instance'] in envioshechos:
        levelsUnique = levelsUnique + [json.loads(envio['contenido'])]
        envioshechos = envioshechos + [envio['instance']]

convergenciasUnique = []
for envio in convergencias:
    if not envio['instance'] in envioshechos:
        convergenciasUnique = convergenciasUnique + [json.loads(envio['contenido'])]
        envioshechos = envioshechos + [envio['instance']]


Definimos la lista de usuarios que hicieron el experimento. Recupere de la compu donde se hicieron quien es quien a partir de las carpetas donde se guardaron los datos

In [38]:
usuarios = {459443370368:'Agustin', 1460500029182:'Chalukian', 1460568434768:'Chechu',1459459628908:'Euge',
           1459455977180:'Flor', 1459450579229:'Guido', 1460384100961:'Jule', 1459368818910:'Lungo',
           1460499050987:'Marta', 1459358889070:'Martina', 1460645753225:'Pierina', 1459350343607:'Roli'}

142

In [53]:
convergenciasUnique[0]['expLog']['session']

{'codeVersion': 30,
 'plataforma': 'Desktop',
 'resourcesVersion': 1,
 'sessionInstance': 1459264438837,
 'userId': 1459264438868}